In [1]:
'''
!pip3 install dash
!pip3 install jupyter_dash
!pip3 install numpy
!pip3 install pandas
!pip3 install dash_bootstrap_components
!pip3 install sklearn
!pip3 install osmnx
!pip3 install utm
!pip3 uninstall umap
!pip3 install umap-learn -Y
'''

'''
conda config --prepend channels conda-forge
conda create -n ox --strict-channel-priority osmnx jupyter
conda activate ox
conda install -c conda-forge dash
conda install -c plotly jupyter-dash
conda install -c conda-forge dash-bootstrap-components
conda install -c conda-forge utm
conda install -c conda-forge umap-learn
'''

'\nconda config --prepend channels conda-forge\nconda create -n ox --strict-channel-priority osmnx jupyter\nconda activate ox\nconda install -c conda-forge dash\nconda install -c plotly jupyter-dash\nconda install -c conda-forge dash-bootstrap-components\nconda install -c conda-forge utm\nconda install -c conda-forge umap-learn\n'

In [1]:
import os
import CityHub
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import math
from statistics import mean
from sklearn.manifold import TSNE

In [2]:
# carrega o citygraph preprocessado
ch=CityHub.load_preprocessed_CityHub('data/sp-sibgrapi2022.bin')

In [3]:
#carrega o array x de features estáticas e a tabela de indexação de esquinas dos arquivos csv

X_static = pd.read_csv("data/embeddings_12features.csv", header=None)
tabela_esquinas_bbox = list(X_static.index)
X_static

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,5.736410e+05,5.407892,-5.501085,-1.001470,-8.059328,-0.880598,2.440312,1.197996,4.843651,-1.602810,0.240249,1.330662,3.407556
1,5.736430e+05,-0.819824,-0.521442,0.333270,-0.479956,0.137803,1.989848,0.133119,1.283837,-1.069944,0.629820,-0.689618,0.680249
2,5.736440e+05,0.445507,0.090402,0.240793,-0.324101,-0.083868,-0.047367,-0.117901,0.162269,-0.086959,0.042288,0.213293,0.179955
3,5.772390e+05,0.656317,0.047656,0.816010,-0.744200,-0.150205,-0.246325,-0.196421,-0.367433,0.276700,-0.117257,-0.144837,-0.312913
4,5.821390e+05,1.299440,-0.164971,0.480875,-0.495205,-0.686019,-0.343559,0.496338,0.172237,-0.123124,-0.236612,0.074542,0.295032
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119743,8.851304e+09,-0.127382,0.040168,0.481595,-0.460615,-0.139547,-0.202971,0.123555,-0.526503,0.675220,0.142232,0.432332,0.413546
119744,8.851395e+09,0.071168,0.207792,0.036443,0.115815,-0.054424,-0.133884,-0.038442,-0.275935,0.148837,-0.141205,-0.217031,0.104510
119745,8.851395e+09,-0.087780,0.174925,0.115053,-0.692299,-0.452189,-0.355414,0.486273,0.160699,0.466169,-0.188006,-0.108264,0.752105
119746,8.866803e+09,0.386760,0.267389,-0.385811,-0.290987,0.132366,0.536733,0.024995,0.142566,-0.316761,-0.068932,-0.080925,-0.014172


In [4]:
# gerando df para com features estáticas e dinâmicas
# é possível incluir initial_date e final_date em compute_dynamic_feature_df
# df = pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,'2020-01-01','2020-12-31')], axis = 1)
df = X_static.copy()
static_feature_names = X_static.columns[0:13]
dynamic_feature_names = []

In [5]:
# Redução de dimensionalidade UMAP, a primeira com todas as features
import umap.umap_ as umap
reducer = umap.UMAP()
df_normalized=(df-df.mean())/df.std()
df_normalized=df_normalized.fillna(0)
X_embedded = reducer.fit_transform(df_normalized)

In [6]:
df['lat'] =[ch.city_vert_list[i][0] for i in tabela_esquinas_bbox]
df['lon'] = [ch.city_vert_list[i][1] for i in tabela_esquinas_bbox]
df['u'] = X_embedded[:,0]
df['v'] = X_embedded[:,1]

In [7]:
# Calcule histogramas e distâncias entre o subconjunto selecionado e o total
def compute_histograms(df_name, nbins = 18):
    hists = dict()
    bins = dict()
    for col in df_name.columns[:-4]:
        hists[col],bins[col]=np.histogram(df_name[col], bins=nbins, density=True)
    return hists,bins

In [8]:
full_hists,full_bins = compute_histograms(df)

In [9]:
def compute_histograms_definedbins(df_name, definedbins):
    hists = dict()
    bins = dict()
    for col in df_name.columns[:-4]:
        hists[col],bins[col]=np.histogram(df_name[col], bins=definedbins[col], density=True)
    return hists,bins

In [10]:
partial_hists,partial_bins = compute_histograms_definedbins(df,full_bins)

In [11]:
# calculate the kl divergence
#def kl_divergence(p, q):
#   return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
from scipy.stats import wasserstein_distance

def kl(p, q):
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0.0))


def hist_euc(p, q):
    return np.linalg.norm(p/np.linalg.norm(p)-q/np.linalg.norm(q))


def dict_divergence(dict1,dict2):
    dict_div = dict()
    for k, v in dict1.items():
        dict_div[k]=hist_euc(v,dict2[k])    
    #   dict_div[k]=wasserstein_distance(v,dict2[k])
    #    if k=='Crimes':
    #    print(dict_div[k])
    return dict_div

In [12]:
df_backup=df
df_backup

,0,1,2,3,4,5,6,7,8,9,10,11,12,lat,lon,u,v
0,5.736410e+05,5.407892,-5.501085,-1.001470,-8.059328,-0.880598,2.440312,1.197996,4.843651,-1.602810,0.240249,1.330662,3.407556,-23.562991,-46.703965,3.796774,8.816181
1,5.736430e+05,-0.819824,-0.521442,0.333270,-0.479956,0.137803,1.989848,0.133119,1.283837,-1.069944,0.629820,-0.689618,0.680249,-23.567465,-46.702229,2.528740,7.341290
2,5.736440e+05,0.445507,0.090402,0.240793,-0.324101,-0.083868,-0.047367,-0.117901,0.162269,-0.086959,0.042288,0.213293,0.179955,-23.572962,-46.699235,-1.682301,8.570189
3,5.772390e+05,0.656317,0.047656,0.816010,-0.744200,-0.150205,-0.246325,-0.196421,-0.367433,0.276700,-0.117257,-0.144837,-0.312913,-23.571831,-46.702075,-2.290476,6.800118
4,5.821390e+05,1.299440,-0.164971,0.480875,-0.495205,-0.686019,-0.343559,0.496338,0.172237,-0.123124,-0.236612,0.074542,0.295032,-23.534616,-46.748148,-0.253302,8.345157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119743,8.851304e+09,-0.127382,0.040168,0.481595,-0.460615,-0.139547,-0.202971,0.123555,-0.526503,0.675220,0.142232,0.432332,0.413546,-23.649217,-46.667740,9.807174,4.673246
119744,8.851395e+09,0.071168,0.207792,0.036443,0.115815,-0.054424,-0.133884,-0.038442,-0.275935,0.148837,-0.141205,-0.217031,0.104510,-23.651471,-46.662832,11.054634,5.393978
119745,8.851395e+09,-0.087780,0.174925,0.115053,-0.692299,-0.452189,-0.355414,0.486273,0.160699,0.466169,-0.188006,-0.108264,0.752105,-23.651152,-46.663291,10.048100,5.013996
119746,8.866803e+09,0.386760,0.267389,-0.385811,-0.290987,0.132366,0.536733,0.024995,0.142566,-0.316761,-0.068932,-0.080925,-0.014172,-23.500164,-46.611595,9.884541,6.179490


In [13]:
import dash
from dash import dcc, ctx
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pddict_divergence
import plotly.express as px
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
from datetime import date

#app = JupyterDash()
selected_corners_indices = []
app = JupyterDash(external_stylesheets=[dbc.themes.PULSE])
server = app.server
app.index_string = """
<!DOCTYPE html>
<html>
   <head>
       {%metas%}
       <title>{%title%}</title>
       {%favicon%}
       {%css%}
       <style>

            .DateInput_input__focused{
                border-bottom: 2px solid #0175ff;
            }
            .DateInput_input, .DateInput_input_1 {
                font-family: Arial;
                font-size: 10px; 
                height: 10px;
            }
            .DateInput, .DateInput_1 {
                width: 40%;
                height: inherit;
            }
       </style>
   </head>
   <body>
       {%app_entry%}
       <footer>
           {%config%}
           {%scripts%}
           {%renderer%}
       </footer>
   </body>
</html>
"""
df=df_backup.copy()
# Configuração de janela padrão
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name = df.index, hover_data=df.columns[:-2],
                        color_discrete_sequence=['#00a699'], zoom=3, height=300)#"fuchsia"
fig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )

scatter = go.Figure()
scatter.add_trace(go.Scattergl(
                x = df.u,
                y = df.v,
                mode = 'markers',
                marker = dict(
                color='#00a699', #green 
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
))
scatter.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    dragmode='select',
    clickmode='event+select')

dict_div=dict_divergence(full_hists,full_hists)
data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
fig_barplot = go.Figure(data=data)
fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
fig_barplot.update_xaxes(tickfont_size=10)

def blank_fig(text):
    fig = go.Figure(go.Scatter(x=[], y = []))
    fig.update_layout(
          title_text=text,title_font_size=10,
        template = None,height=30)
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_layout({"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"})
    return fig

# Estrutura das janelas
app.layout = dbc.Container(
    [
        
        html.H1(),
        html.Center(html.H4(children='CityHub Visual Explorer')),
        
        html.Hr(),
         dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features')),
                        dbc.Row(html.H6(children='Static')), 
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_static",
                               options=static_feature_names,
                               value=static_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        dbc.Row(html.H6(children='Dynamic')), 
                        dbc.Row(
                            dcc.DatePickerRange(
                                id='my-date-picker-range',
                                start_date=date(2020, 1, 1),
                                end_date=date(2020, 12, 31),
                                min_date_allowed = date(2020, 1, 1),
                                max_date_allowed = date(2020, 12, 31),
                            )
                        ),
                        html.H2(),
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_dynamic",
                               options=dynamic_feature_names,
                               value=dynamic_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        html.H2(),
                        dbc.Button(
                            str('Update'),
                            className="btn btn-primary",
                            id='update_button',
                            n_clicks=0,
                        )
                        ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Projection View')),                     
                        dbc.Row(dcc.Graph(id="projectionScatter", figure=scatter)),
                            ], width=5),
                    dbc.Col([
                        dbc.Row(html.H5(children='Map View')),
                        dbc.Row(dcc.Graph(id="mymap", figure=fig))
                        ], width=5),
                ]
            ),
        dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features distribution')),  
                        dbc.Row(html.H6(children='Global')), 
                        dbc.Row(dcc.Graph(id="general",figure= blank_fig(''))),
                        dbc.Row(html.H6(children='Selected')),  
                        dbc.Row(dcc.Graph(id="detail",figure= blank_fig(''))),
                            ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Feature Histogram Distance')),                     
                        dbc.Row(dcc.Graph(id="barplot", figure=fig_barplot)),
                            ], width=10),
                ]
            ),
        dcc.Store(id="features-detail-df"),
        dcc.Store(id="features-general-df"),
    ]
)


# Projection View
def scatterplot(x, y, title="", axis_type="Linear"):
    return {
        "data": [go.Scatter(x=df.u, y=df.v, mode="markers") ],
    }

# @app.callback atualiza as visualizações, tem entrada, saída e função
@app.callback(
    Output("barplot", "figure"),
    Output("projectionScatter", "figure"),
    Output("mymap", "figure"),
    Output('features-detail-df', 'data'),
    Output('features-general-df', 'data'),
    [
        Input("features_checklist_static", "value"),
        Input("features_checklist_dynamic", "value"),
        Input('my-date-picker-range', 'start_date'),
        Input('my-date-picker-range', 'end_date'),
        Input("projectionScatter", "selectedData"),
        Input("projectionScatter", "figure"),
        Input("barplot", "figure"),
        Input("mymap", "selectedData"),
        Input("mymap", "figure"),
        Input('features-detail-df', 'data'),
        Input('features-general-df', 'data'),
        Input('update_button', 'n_clicks'),
    ], prevent_initial_call=True
)
def update_cb(options_chosen_static, options_chosen_dynamic, start_date, end_date, selectedData, scatter_current_fig, barplot_current_fig, selectedDataMap, map_current_fig,features_current_data,features_current_data_general,n_clicks):
    triggered_id = ctx.triggered_id
    global selected_corners_indices
    if triggered_id == 'mymap':
        if len(selectedDataMap["points"])==0:
            return barplot_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general
        # Se você tiver features dinâmicos, eles deverão ser calculados e concatenados
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        temp=df2
        pt = selectedDataMap["points"]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt:
            #indices.append(sel['hovertext'])
            selected_corners_indices.append(sel['hovertext'])
        selected_corners_indices = list(set(selected_corners_indices))
        #temp=temp.loc[indices]
        temp=temp.iloc[selected_corners_indices]
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df.u,#scatter_current_fig['data'][0]['x'],
                y = df.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        return fig_barplot,scatter,map_current_fig,temp.to_json(date_format='iso', orient='split'), df2.to_json(date_format='iso', orient='split')

    elif triggered_id == 'projectionScatter':
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        #options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2
        temp["lat"] = df["lat"]
        temp["lon"] = df["lon"]
        pt = pd.DataFrame(selectedData["points"])
        pt = pt[pt.curveNumber == 0]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt['pointIndex']:
            #indices.append(sel['pointIndex'])
            selected_corners_indices.append(sel)
        #temp=temp.iloc[indices]
        selected_corners_indices = list(set(selected_corners_indices))
        temp=temp.iloc[selected_corners_indices]
        mapfig = px.scatter_mapbox(temp, lat="lat", lon="lon", hover_name=temp.index, hover_data=temp.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#'blue'
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df2.u,#scatter_current_fig['data'][0]['x'],
                y = df2.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        return fig_barplot,scatter,mapfig,temp.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    
    elif triggered_id == 'update_button': #or triggered_id =='features_checklist_static' or triggered_id == 'features_checklist_dynamic' or triggered_id == 'my-date-picker-range':
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        print(df2.columns)
        options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2.filter(options_chosen)
        temp_normalized=(temp-temp.mean())/temp.std()
        temp_normalized.fillna(0)
        X_embedded = reducer.fit_transform(temp_normalized)
        temp['u'] = X_embedded[:,0]
        temp['v'] = X_embedded[:,1]
        df.u=temp.u
        df.v=temp.v
        df2.u=temp.u
        df2.v=temp.v
        selected_df = df2.iloc[selected_corners_indices]
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = selected_df.u,
                y = selected_df.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend = False)       
        mapfig = px.scatter_mapbox(selected_df, lat="lat", lon="lon", hover_name=selected_df.index,  hover_data=selected_df.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#"blue"
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450,
                 )  
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(selected_df,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)
        
        return fig_barplot,scatter,mapfig,selected_df.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    else:
        return barplot_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general

# Atualiza Features distribution View (Global y Selected) ao clicar nas barras
@app.callback(
    Output('detail', 'figure'),
    Output('general', 'figure'),
    Input('barplot', 'clickData'),
    Input('features-detail-df', 'data'),
    Input('features-general-df', 'data'),
    Input('detail', 'figure'),
    Input('general', 'figure'),
    
    prevent_initial_call=True
)
def display_click_data(clickData,df_json, df_global_json, detail_current_fig, general_current_fig):
    if clickData is None:
        return detail_current_fig, general_current_fig
    
    detail_df = pd.read_json(df_json, orient='split')    
    geral_df = pd.read_json(df_global_json, orient='split')
    name = clickData['points'][0]['label']
    try:
        range_min = geral_df[name].min()
        range_max = geral_df[name].max()
    except:
        range_min = 100000
        range_max = -1
 
    #detail_df = detail_df.groupby(name).count().reset_index()
    #geral_df = geral_df.groupby(name).count().reset_index()
    
    if len(geral_df[name].unique().tolist()) > 12:
        #yy=[i for i in range(0, len(xx))]
        geral_xx = geral_df[name].tolist()
        detail_xx = detail_df[name].tolist()
        step = (range_max-range_min)/10
        fig = go.Figure([go.Histogram(x=detail_xx,xbins=dict(size=step))])
        fig.update_xaxes(range=[range_min-step, range_max+step])
        fig.update_layout(bargap=0.3)
        
        geral_fig = go.Figure([go.Histogram(x=geral_xx,xbins=dict(size=step))])
        geral_fig.update_xaxes(range=[range_min-step, range_max+step])
        geral_fig.update_layout(bargap=0.3)
    else: 
        geral_df = geral_df.groupby(name).count().reset_index()
        detail_df = detail_df.groupby(name).count().reset_index()
        
        geral_xx = geral_df[name].tolist()
        detail_xx = detail_df[name].tolist()
        
        geral_yy = geral_df[geral_df.columns[1]].tolist()
        detail_yy = detail_df[detail_df.columns[1]].tolist()
       
        yyy = []
        num = 0
        for i in geral_xx:
            if i in detail_xx:
                yyy.append(detail_yy[num])
                num +=1
            else:
                yyy.append(0)
        common_x = np.array(geral_xx).astype(str)
        fig = go.Figure([go.Bar(x=common_x, y=yyy)])
        geral_fig = go.Figure([go.Bar(x=common_x, y=geral_yy)])
    
    fig.update_layout(title_text=name,title_font_size=10,
            margin ={'l':0,'t':20,'b':0,'r':0},height=150)
    geral_fig.update_layout(title_text=name,title_font_size=10,
            margin ={'l':0,'t':20,'b':0,'r':0},height=150)
    return fig,geral_fig

# Run server
if __name__ == "__main__":
   # app.run_server(debug=True,mode="inline")
    app.run_server()
    #df=df_backup

Dash app running on http://127.0.0.1:8050/
